In [1]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-learn >0.20 é necessário
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version 2.x  # Este comando é usado apenas no Google Colab
    pass
except Exception:
    pass

# Tensorflow >2.0 é necessário
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Carregamento do TensorBoard
try:
    from IPython import get_ipython
    get_ipython().run_line_magic('load_ext', 'tensorboard')
except:
    print("TensorBoard não pôde ser carregado. Certifique-se de que está em um ambiente compatível, como Jupyter Notebook ou Google Colab.")

import numpy as np
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandálias ou camisas
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # indices das classes 7, 8, 9 deveriam ser alterados para 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # classificação binaria: é uma camisa? (classe 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]



In [3]:
X_train_A.shape

(43986, 28, 28)

In [4]:
X_train_B.shape

(200, 28, 28)

In [5]:
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

c:\Users\lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [8]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7449 - loss: 0.7922 - val_accuracy: 0.8665 - val_loss: 0.3817
Epoch 2/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8722 - loss: 0.3640 - val_accuracy: 0.8857 - val_loss: 0.3334
Epoch 3/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8848 - loss: 0.3257 - val_accuracy: 0.8936 - val_loss: 0.3125
Epoch 4/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8909 - loss: 0.3067 - val_accuracy: 0.8951 - val_loss: 0.3003
Epoch 5/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8957 - loss: 0.2941 - val_accuracy: 0.8981 - val_loss: 0.2916
Epoch 6/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8993 - loss: 0.2846 - val_accuracy: 0.8986 - val_loss: 0.2849
Epoch 7/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9028 - loss: 0.2769 - val_accuracy: 0.8986 - val_loss: 0.2803
Epoch 8/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9055 - loss: 0.2703 - 

In [9]:
model_A.save("my_model_A.keras")

In [10]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [11]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [12]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.3347 - loss: 0.8267 - val_accuracy: 0.6095 - val_loss: 0.6549
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6199 - loss: 0.6256 - val_accuracy: 0.7130 - val_loss: 0.5460
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7532 - loss: 0.5127 - val_accuracy: 0.8002 - val_loss: 0.4647
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8492 - loss: 0.4288 - val_accuracy: 0.8702 - val_loss: 0.4032
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9031 - loss: 0.3661 - val_accuracy: 0.9026 - val_loss: 0.3553
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9156 - loss: 0.3180 - val_accuracy: 0.9209 - val_loss: 0.3175
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9439 - loss: 0.2801 - val_accuracy: 0.9402 - val_loss: 0.2869
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9508 - loss: 0.2498 - val_accuracy: 0.9483 - val_loss: 0.2618
Epoch 9

In [13]:
model_A = keras.models.load_model("my_model_A.keras")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [14]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [15]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4034 - loss: 1.9149 - val_accuracy: 0.4047 - val_loss: 1.7776
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4184 - loss: 1.7895 - val_accuracy: 0.4270 - val_loss: 1.6590
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4391 - loss: 1.6690 - val_accuracy: 0.4432 - val_loss: 1.5457
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4568 - loss: 1.5538 - val_accuracy: 0.4594 - val_loss: 1.4380
Epoch 5/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4637 - loss: 1.4442 - val_accuracy: 0.4848 - val_loss: 1.3360
Epoch 6/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4746 - loss: 1.3404 - val_accuracy: 0.5041 - val_loss: 1.2399
Epoch 7/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4991 - loss: 1.2426 - val_accuracy: 0.5233 - val_loss: 1.1498
Epoch 8/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5205 - loss: 1.1508 - val_accuracy: 0.5426 - val_loss: 1.0657
Epoch 9

In [16]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7565 - loss: 0.4910 - val_accuracy: 0.9300 - val_loss: 0.2796
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9411 - loss: 0.2477 - val_accuracy: 0.9544 - val_loss: 0.2121
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9569 - loss: 0.1848 - val_accuracy: 0.9665 - val_loss: 0.1767
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9595 - loss: 0.1500 - val_accuracy: 0.9736 - val_loss: 0.1528
Epoch 5/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9773 - loss: 0.1262 - val_accuracy: 0.9767 - val_loss: 0.1354
Epoch 6/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9892 - loss: 0.1089 - val_accuracy: 0.9797 - val_loss: 0.1222
Epoch 7/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9931 - loss: 0.0959 - val_accuracy: 0.9807 - val_loss: 0.1117
Epoch 8/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9931 - loss: 0.0858 - val_accuracy: 0.9807 - val_loss: 0.1033
Epoch 9

In [17]:
model_B.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.9793 - loss: 0.1264


[0.13018961250782013, 0.9779999852180481]

In [18]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.9946 - loss: 0.0613


[0.06250172108411789, 0.9940000176429749]